In [12]:
!pip install pymagnitude git+https://github.com/huggingface/pytorch-pretrained-BERT.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [13]:
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import random
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex as re
#from pymagnitude import Magnitude
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from sklearn.metrics import classification_report
from tqdm import tqdm, tqdm_notebook
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from IPython.core.display import display, HTML
from google_drive_downloader import GoogleDriveDownloader as gdd

tqdm.pandas()

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [15]:
data = pd.read_csv('/content/train.tsv', sep='\t')
data_val = pd.read_csv('/content/validation.tsv', sep='\t')
data_test = pd.read_csv('/content/test.tsv', sep='\t')

In [16]:
# convert to lowercase
data['clean_text']=data['Tweet'].str.lower()
#remove URLS
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
#remove punctuations
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
#remove empty lines
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
#remove digits
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
#remove multiple spaces
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
#remove single character
data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))

In [17]:
data_val

,Tweet,Claim,Stance,Premise
0,Ordered a mask that had a cute chain attached ...,face masks,FAVOR,0
1,Who is ready for some #baseball? #BaseballisB...,stay at home orders,NONE,0
2,"'@Mystere07623203 @va_shiva @Liberty13046 130,...",face masks,FAVOR,1
3,When they ask me what I did with my life I wil...,face masks,FAVOR,0
4,Taylor not putting her album in physical store...,stay at home orders,FAVOR,0
...,...,...,...,...
595,@BrianKempGA is one of the few #Governors that...,face masks,AGAINST,1
596,'I tested positive for Covid. Got it from a fr...,face masks,FAVOR,1
597,"'If you are under retirement age, you have a *...",stay at home orders,AGAINST,1
598,'@TheLalasventure @doqholliday @realDonaldTrum...,face masks,FAVOR,0


In [18]:
data_test

,id,claim,text
0,1307558525371965442,school closures,@narendramodi @rajnathsingh Student ka bhi soa...
1,1247739239879467009,stay at home orders,—échale un vistazo a esto… … a fair piece on...
2,1242046510155653125,stay at home orders,Why do think skilling women and girls is impor...
3,1358446499949084675,school closures,To reduce the risk of the virus spreading as e...
4,1249740062775902208,stay at home orders,I speak for a great many people when i say WE ...
...,...,...,...
9950,1242516037628813314,stay at home orders,StayAtHomeSaveLives 21daysLockdown StayAtHome ...
9951,1242746919933415424,stay at home orders,If this is true this is heartbreaking StayAtHo...
9952,1276638598813679617,stay at home orders,"855 Sunset Cove Dr, Winter Haven, FL 33880 3 B..."
9953,1243504288661270528,stay at home orders,StayAtHomeSaveLives StayHomeStaySafe StayHome ...


In [19]:
import re
def clean_text(text):
    text = re.sub("'", "", text)
    text = re.sub("(\\W)+", " ", text)
    text = text.lower()
    return text

data['train'] = data['clean_text'].apply(clean_text)
DATA_PATH = data['train']
train_dataset = data['train']
valid_dataset = data_val['Tweet']
test_dataset = data_test['text']
#data_val['val'] = data_val['clean_text'].apply(clean_txt)


In [20]:
class SentimentDataset(Dataset):
    def __init__(self, data_path, max_len):
        df = data

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        df['tokenized_text'] = df.train.progress_apply(self.tokenizer.tokenize)
        
        # Shorten to max length (Bert has a limit of 512); subtract two tokens for [CLS] and [SEP]
        df.loc[:, 'tokenized_text'] = df.tokenized_text.str[:max_len - 2]
        
        # Add Bert-specific beginning and end tokens
        df.loc[:, 'tokenized_text'] = df.tokenized_text.apply(
            lambda tokens: ['[CLS]'] + tokens + ['[SEP]'],
        )
        
        df['indexed_tokens'] = df.tokenized_text.progress_apply(
            self.tokenizer.convert_tokens_to_ids,
        )
        
        sequences = df.indexed_tokens.tolist()
        max_sequence_length = max(len(x) for x in sequences)
        
        self.inputs_lst, self.masks, self.segments = [], [], []
        for sequence in sequences:
            self.inputs_lst.append(sequence + (max_sequence_length - len(sequence)) * [0])
            self.masks.append(len(sequence) * [1] + (max_sequence_length - len(sequence)) * [0])
            self.segments.append(max_sequence_length * [0])
            
        self.targets = df.Premise.tolist()
        self.texts = df.train.tolist()

    def __getitem__(self, i):
        return self.inputs_lst[i], self.masks[i], self.segments[i], self.targets[i], self.texts[i]
    
    def __len__(self):
        return len(self.inputs_lst)

In [21]:
# How many tokens long each sequence will be cut to
# Shorter sequences will get the padding token <PAD>
max_len = 128  #@param {type:"slider", min:16, max:512, step:2}

dataset = SentimentDataset(DATA_PATH, max_len)
dataset_val = SentimentDataset(valid_dataset, max_len)
dataset_test = SentimentDataset(test_dataset, max_len)

100%|██████████| 3556/3556 [00:00<00:00, 54788.23it/s]


In [22]:
#@title How many examples to load on the GPU at once

def collate(batch):
    inputs = torch.LongTensor([item[0] for item in batch])
    mask = torch.LongTensor([item[1] for item in batch])
    segment = torch.LongTensor([item[2] for item in batch])
    target = torch.LongTensor([item[3] for item in batch])
    text = [item[4] for item in batch]
    
    inputs, mask, segment, target = map(
        lambda x: x.to(device),
        (inputs, mask, segment, target),
    )

    return inputs, mask, segment, target, text

batch_size = 32  #@param {type:"integer"}

train_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate)
valid_loader = DataLoader(dataset_val, batch_size=batch_size, collate_fn=collate)
test_loader = DataLoader(dataset_test, batch_size=batch_size, collate_fn=collate)

## Train the model

We will train the smaller model `bert-base-uncased` as the larger one has problems fitting into memory. A multi-GPU setup is likely needed for training the larger model. You can find the full list of possible models [here](https://github.com/huggingface/pytorch-pretrained-BERT#loading-google-ai-or-openai-pre-trained-weigths-or-pytorch-dump).

- **Learning rate**: How quickly the model learns. If it's too low, the model will learn very slowly. If it's too high, the model won't be able to learn at all. The goal is to have as high as value as we can get away with.

Note that learning rate should be lower than what you maybe are used to (eg. 1e-5 instead of 1e-3), as we are only fine-tuning the BERT model instead of training it from scratch. We don't want the model's parameters to drastically change as that will likely result in a decrease in model performance.

In [23]:
def train_epoch(model, optimizer, train_loader):
    model.train()
    train_loss = total = 0
    for inputs, mask, segment, target, text in notebook.tqdm(train_loader,
                                                             desc='Training',
                                                             leave=False):
        optimizer.zero_grad()
        
        loss = model(inputs, segment, mask, target)

        train_loss += loss.item()
        total += 1
        
        loss.backward()
        optimizer.step()
        
    return train_loss / total


def validate_epoch(model, valid_loader):
    model.eval()
    with torch.no_grad():
        valid_loss = total = 0
        for inputs, mask, segment, target, text in notebook.tqdm(valid_loader,
                                                                 desc='Validating',
                                                                 leave=False):
            loss = model(inputs, segment, mask, target)

            valid_loss += loss.item()
            total += 1

        return valid_loss / total

In [24]:
#@title How big of steps the model takes while learning

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = model.to(device)

learning_rate = 0.001  #@param {type:"number"}

param_optimizer = list(model.classifier.named_parameters()) 
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer]}]
optimizer = BertAdam(optimizer_grouped_parameters, lr=learning_rate)

100%|██████████| 407873900/407873900 [00:33<00:00, 12182337.97B/s]
t_total value of -1 results in schedule not being applied


In [25]:
#@title Limit the number of training epochs (training is slow)

max_epochs = 3  #@param {type:"slider", min:1, max:10}

n_epochs = 0
train_losses, valid_losses = [], []


## Predictions

In [ ]:
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, mask, segment, target, text in test_loader:
        loss = model(inputs, segment, mask, target)
        logits = model(inputs, segment, mask)
        
        logits = logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=1)
        target = target.cpu().numpy()
        
        y_true.extend(predictions)
        y_pred.extend(target)
        
print(classification_report(y_true, y_pred))

In [ ]:
y_true

In [ ]:
y_pred

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()